In [33]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import os
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader 
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import numpy as np
from os.path import join
import pandas as pd
from datetime import datetime
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model

In [34]:
hashtags = {'gohawks' : 188136,'nfl' : 259024,'sb49' : 826951,'gopatriots' : 26232,'patriots' : 489713,'superbowl' : 1348767}

In [61]:
def crossvalidation(X, y, model_number, features):
    error = []
    rmse =[]
    for train_ind, test_ind in KFold(n_splits=10).split(X):
        X_train, X_test = X[train_ind], X[test_ind]
        y_train, y_test = y[train_ind], y[test_ind]
        if model_number==0:
            modl = linear_model.Ridge(alpha = .5)
            modl.fit(X_train,y_train)
        elif model_number==1:
            modl = SVR(kernel='rbf', C=1e3, gamma=0.1)
            modl.fit(X_train,y_train)
        elif model_number==2:
            modl = MLPRegressor(activation='relu',solver='adam', alpha=1e-5,
                           hidden_layer_sizes=(features, 1), random_state=1)
            modl.fit(X_train,y_train)
        elif model_number==3:
            modl=RandomForestRegressor(max_depth=features/2, random_state=0)
            modl.fit(X_train,y_train)
        else:
            modl=linear_model.LinearRegression()
            modl.fit(X_train,y_train)
        predction = modl.predict(X_test)
        error.append(mean_absolute_error(y_test, predction))
        rmse.append(sqrt(mean_squared_error(y_test, predction)))
    return error, rmse

In [62]:
def calc_meanerror(X, y, model_number, features):
    if model_number==0:
        modl = linear_model.Ridge(alpha = .5)
        modl.fit(X,y)
    elif model_number==1:
        modl = SVR(kernel='rbf', C=1e3, gamma=0.1)
        modl.fit(X,y)
    elif model_number==2:#non linear
            modl = MLPRegressor(activation='relu',solver='adam', alpha=1e-5,
                           hidden_layer_sizes=(features, 1), random_state=1)
            modl.fit(X,y)
    elif model_number==3:
            modl=RandomForestRegressor(max_depth=features/2, random_state=0)
            modl.fit(X,y)
    else:
        modl=linear_model.LinearRegression()
        modl.fit(X,y)
    predction = modl.predict(X)
    error=mean_absolute_error(y, predction)
    rmse=sqrt(mean_squared_error(y, predction))
    return error,rmse

In [63]:
def get_model(htag, grp, model_number, features):  
    number_tweets=hashtags[htag]
    print("Data for " + str(htag) + " hashtag")
    #prepare to read each file
    with open("tweet_data/" + "".join('tweets_#' + htag + '.txt'), 'r') as fl:
        datafr = pd.DataFrame(index=range(number_tweets), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
            'favoriteCount','longTweets', 'rankingScore', 'userID'])
        cnt = 0
        for line in fl:
            tweet = json.loads(line)
            date = datetime.fromtimestamp(tweet['firstpost_date'])
            datafr.set_value(cnt, 'Time', date)
            datafr.set_value(cnt, 'tweetCount', 1)
            datafr.set_value(cnt, 'retweetCount', tweet['metrics']['citations']['total'])
            datafr.set_value(cnt, 'followerSum', tweet['author']['followers'])
            datafr.set_value(cnt, 'maxFollowers', tweet['author']['followers'])
            datafr.set_value(cnt, 'location', tweet['tweet']['user']['location'])
            datafr.set_value(cnt, 'impressionCount', tweet['metrics']['impressions'])
            datafr.set_value(cnt, 'favoriteCount', tweet['tweet']['favorite_count'])
            datafr.set_value(cnt, 'longTweet', len(tweet.get('title')) > 100)
            datafr.set_value(cnt, 'rankingScore', tweet.get('metrics').get('ranking_score'))
            datafr.set_value(cnt, 'userID', tweet.get('tweet').get('user').get('id'))
            cnt += 1

        start_date=datetime(2015,2,1,8,0,0)
        end_date=datetime(2015,2,1,20,0,0)
        if prd==1:
            df_start= datafr[datafr.Time<start_date]
            print("Before Feb. 1, 8:00 a.m.")
            X,y=hourly_data(df_start,features)
        elif prd==2:
            df_mid = datafr[(datafr.Time>start_date) & (datafr.Time<end_date)]
            print("Between Feb. 1, 8:00 a.m. and 8:00 p.m.")
            X,y=hourly_data(df_mid,features)
        else:
            df_end = datafr[datafr.Time>end_date]
            print("After Feb. 1, 8:00 p.m")
            X,y=hourly_data(df_end,features)
    
    if model_number==0:
        modl = linear_model.Ridge(alpha = .5)
        modl.fit(X,y)
    elif model_number==1:
        modl = SVR(kernel='rbf', C=1e3, gamma=0.1)
        modl.fit(X,y)
    elif model_number==2:#non linear
        modl = MLPRegressor(activation='relu',solver='adam', alpha=1e-5,
            hidden_layer_sizes=(features, 1), random_state=1)
        modl.fit(X,y)
    elif model_number==3:
            modl=RandomForestRegressor(max_depth=features/2, random_state=0)
            modl.fit(X,y)
    else:
        modl=linear_model.LinearRegression()
        modl.fit(X,y)
    return modl

In [64]:
def hourly_data(datafr, features):
    datafr = datafr.set_index('Time')
    group_hour = datafr.groupby(pd.TimeGrouper(freq='60Min'))
    X=np.ones((len(group_hour),features))
    y=np.ones((len(group_hour),1))
    for cnt,(interval,grp) in enumerate(group_hour):
        y[cnt,0]=grp.tweetCount.sum()
        tweetCount = grp.tweetCount.sum()
        tweetCount = max(1, tweetCount)
        X[cnt, 0] = interval.hour                 # Hour of the day
        X[cnt, 1] = tweetCount                   # Number of tweets
        X[cnt, 2] = grp.retweetCount.sum()      # Number of retweets
        X[cnt, 3] = grp.followerSum.sum()       # Sum of followers in this interval
        X[cnt, 4] = grp.maxFollowers.max()      # Max following in this interval
        X[cnt, 5] = grp.location.value_counts().max()
        X[cnt, 6] = grp.impressionCount.sum() / tweetCount   # Sum of impression count
        X[cnt, 7] = grp.favoriteCount.sum() / tweetCount    # Sum of favorites
        X[cnt, 8] = grp.longTweet.sum()         # Number of long tweets
        X[cnt, 9] = grp.rankingScore.sum() / tweetCount
        X[cnt, 10] = grp.userID.nunique()
        
    y=y[1:]
    X=np.nan_to_num(X[:-1])    
    return X, y

def six_hour(datafr, features):
    datafr = datafr.set_index('Time')
    group_month = datafr.groupby(pd.TimeGrouper(freq='60Min'))
    X=np.ones((len(group_month),features))
    y=np.ones((len(group_month),1))
    for cnt,(interval,grp) in enumerate(group_month):
        y[cnt,0]=grp.tweetCount.sum()
        tweetCount = grp.tweetCount.sum()
        tweetCount = max(1, tweetCount)
        X[cnt, 0] = interval.hour                 # Hour of the day
        X[cnt, 1] = tweetCount                   # Number of tweets
        X[cnt, 2] = grp.retweetCount.sum()      # Number of retweets
        X[cnt, 3] = grp.followerSum.sum()       # Sum of followers in this interval
        X[cnt, 4] = grp.maxFollowers.max()      # Max following in this interval
        X[cnt, 5] = grp.location.value_counts().max()
        X[cnt, 6] = grp.impressionCount.sum() / tweetCount   # Sum of impression count
        X[cnt, 7] = grp.favoriteCount.sum() / tweetCount    # Sum of favorites
        X[cnt, 8] = grp.longTweet.sum()         # Number of long tweets
        X[cnt, 9] = grp.rankingScore.sum() / tweetCount
        X[cnt, 10] = grp.userID.nunique()
        
    y=y[5:]
    X=np.nan_to_num(X[:-5])    
    return X, y

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
all_tweetCount = {}
regrNum=["LinearRidge", "SVR", "NeuralNet", "RandomForest", 'LinearReg']
features=12#########CHANGE IT TO 9
result={"LinearRidge":{}, "SVR":{}, "NeuralNet":{}, "RandomForest":{}, 'LinearReg':{}}
def exec_regr(model_number):
    dfstart_arr_mae=[]
    dfmid_arr_mae=[]
    dfend_arr_mae=[]
    dfstart_arr_rmse=[]
    dfmid_arr_rmse=[]
    dfend_arr_rmse=[]
    print("regression type: "+str(regrNum[model_number]))
    htags={}
    for (htag, number_tweets) in hashtags.items():
        htags[htag]={'Before':{'MAE':0, 'RMSE':0}, 'During':{'MAE':0, 'RMSE':0}, 'After':{'MAE':0, 'RMSE':0}} 
        print("Data for " + str(htag) + " hashtag")
        with open("tweet_data/" + "".join('tweets_#' + htag + '.txt'), 'r', encoding="utf8") as fl:
            datafr = pd.DataFrame(index=range(number_tweets), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
            'favoriteCount','longTweets', 'rankingScore', 'userID'])
            users = {}
            cnt = 0
            for line in fl:
                #load each line and 
                tweet = json.loads(line)
                date = datetime.fromtimestamp(tweet['firstpost_date'])
                #create each line to add to the dataframe
                datafr.set_value(cnt, 'Time', date)
                datafr.set_value(cnt, 'tweetCount', 1)
                datafr.set_value(cnt, 'retweetCount', tweet['metrics']['citations']['total'])
                datafr.set_value(cnt, 'followerSum', tweet['author']['followers'])
                datafr.set_value(cnt, 'maxFollowers', tweet['author']['followers'])
                datafr.set_value(cnt, 'location', tweet['tweet']['user']['location'])
                datafr.set_value(cnt, 'impressionCount', tweet['metrics']['impressions'])
                datafr.set_value(cnt, 'favoriteCount', tweet['tweet']['favorite_count'])
                datafr.set_value(cnt, 'longTweet', len(tweet.get('title')) > 100)
                datafr.set_value(cnt, 'rankingScore', tweet.get('metrics').get('ranking_score'))
                datafr.set_value(cnt, 'userID', tweet.get('tweet').get('user').get('id'))
                cnt += 1

            start_date=datetime(2015,2,1,8,0,0)
            end_date=datetime(2015,2,1,20,0,0)

            df_start = datafr[datafr.Time<start_date]
            print("Before Feb. 1, 8:00 a.m.")
            X,y=hourly_data(df_start,features)
            mae, rmse = crossvalidation(X,y, model_number, features)
            print("Mean absolute error: ", np.mean(mae))
            print("RMSE: ", np.sqrt(np.mean(rmse)))
            htags[htag]['Before']['MAE'] = np.mean(mae)
            htags[htag]['Before']['RMSE'] = np.sqrt(np.mean(rmse))
            result[regrNum[model_number]]=htags
            dfstart_arr_mae.append(np.mean(mae))
            dfstart_arr_rmse.append(np.sqrt(np.mean(rmse)))

            df_mid = datafr[(datafr.Time>start_date) & (datafr.Time<end_date)]
            print("Between Feb. 1, 8:00 a.m. and 8:00 p.m.")
            X,y=hourly_data(df_mid,features)
            mae, rmse = crossvalidation(X,y, model_number, features)
            htags[htag]['During']['MAE'] = np.mean(mae)
            htags[htag]['During']['RMSE'] = np.sqrt(np.mean(rmse))
            result[regrNum[model_number]]=htags
            print("Mean absolute error: ", np.mean(mae))
            print("RMSE: ", np.sqrt(np.mean(rmse)))
            dfmid_arr_mae.append(np.mean(mae))
            dfmid_arr_rmse.append(np.sqrt(np.mean(rmse)))

            df_end = datafr[datafr.Time>end_date]
            print("After Feb. 1, 8:00 p.m")
            X,y=hourly_data(df_end,features)
            mae, rmse = crossvalidation(X,y, model_number, features)
            htags[htag]['After']['MAE'] = np.mean(mae)
            htags[htag]['After']['RMSE'] = np.sqrt(np.mean(rmse))
            result[regrNum[model_number]]=htags
            print("Mean absolute error: ", np.mean(mae))
            print("RMSE: ", np.sqrt(np.mean(rmse)))
            dfend_arr_mae.append(np.mean(mae))
            dfend_arr_rmse.append(np.sqrt(np.mean(rmse)))
#     print dfstart_arr_rmse, dfmid_arr_rmse, dfend_arr_rmse
    print(dfstart_arr_mae, dfmid_arr_mae, dfend_arr_mae)
    return min(min(dfstart_arr_mae), min(dfmid_arr_mae), min(dfend_arr_mae)) #min of all hashtags and all periods 
# best_model based on min MAE
models=[]
models.append(exec_regr(0))#linear ridge
models.append(exec_regr(1))#svr
models.append(exec_regr(2))#NN
models.append(exec_regr(3))#random forest
models.append(exec_regr(4))#linear reg
    
print(result)

regression type: LinearRidge
Data for gohawks hashtag


In [40]:
best_model = models.index(min(models))
#print(models)
print(result)

ValueError: min() arg is an empty sequence

In [41]:
print(models)

[]


In [42]:
def bestrunRegression(best_model):
    flag=-1
    for (htag, number_tweets) in hashtags.items():
        flag+=1
        print("Data for " + str(htag) + " hashtag")
        with open("tweet/" + "".join('tweets_#' + htag + '.txt'), 'r', encoding='utf8') as fl:
            #make a datafr that stores the values of interest
            datafr1 = pd.DataFrame(index=range(number_tweets), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
            'favoriteCount','longTweets', 'rankingScore', 'userID'])
            cnt = 0
            for line in fl:
                #load each line and 
                tweet = json.loads(line)
                date = datetime.fromtimestamp(tweet['firstpost_date'])
                #create each line to add to the dataframe
                datafr1.set_value(cnt, 'Time', date)
                datafr1.set_value(cnt, 'tweetCount', 1)
                datafr1.set_value(cnt, 'retweetCount', tweet['metrics']['citations']['total'])
                datafr1.set_value(cnt, 'followerSum', tweet['author']['followers'])
                datafr1.set_value(cnt, 'maxFollowers', tweet['author']['followers'])
                datafr1.set_value(cnt, 'location', tweet['tweet']['user']['location'])
                datafr1.set_value(cnt, 'impressionCount', tweet['metrics']['impressions'])
                datafr1.set_value(cnt, 'favoriteCount', tweet['tweet']['favorite_count'])
                datafr1.set_value(cnt, 'longTweet', len(tweet.get('title')) > 100)
                datafr1.set_value(cnt, 'rankingScore', tweet.get('metrics').get('ranking_score'))
                datafr1.set_value(cnt, 'userID', tweet.get('tweet').get('user').get('id'))
                cnt += 1
            if flag==0:
                datafr = pd.concat([datafr1])
            else: datafr = pd.concat([datafr, datafr1])
    #     once all hashtags are in one data frame
    #     print datafr.iloc[288135,1]
    start_date=datetime(2015,2,1,8,0,0)
    end_date=datetime(2015,2,1,20,0,0)

    df_start = datafr[datafr.Time<start_date]
    print("Before Feb. 1, 8:00 a.m.")
    X,y=hourly_data(df_start,features)
    #WIDE DATA FORMAT HERE
    mae, rmse = calc_meanerror(X,y, best_model,features)
    print("Mean absolute error: ", mae)
    print("RMSE: ", rmse)

    df_mid = datafr[(datafr.Time>start_date) & (datafr.Time<end_date)]
    print("Between Feb. 1, 8:00 a.m. and 8:00 p.m.")
    X,y=hourly_data(df_mid,features)
    mae, rmse = calc_meanerror(X,y, best_model,features)
    print("Mean absolute error: ", mae)
    print("RMSE: ", rmse)

    df_end = datafr[datafr.Time>end_date]
    print("After Feb. 1, 8:00 p.m")
    X,y=hourly_data(df_end,features)
    mae, rmse = calc_meanerror(X,y, best_model,features)
    print("Mean absolute error: ", mae)
    print("RMSE: ", rmse)
    
bestrunRegression(best_model)

Data for gohawks hashtag
Data for nfl hashtag
Data for sb49 hashtag
Data for gopatriots hashtag
Data for patriots hashtag


KeyboardInterrupt: 

In [43]:
import nltk

def tokenize(text, tofind):
    text = re.sub(r'[^A-Za-z]', " ", text)
    text = "".join([ch for ch in text if ch not in string.punctuation])
    text = re.sub("[,.-:/()?{}*$&\;!@%]^><~", " ", text)
    text = "".join(ch for ch in text if ord(ch) < 128)
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    return findWord(tokens)

def findWord(tokens, tofind):
    if tofind in tokens:
        return 

In [45]:
mapping={}
def getHashtag(filename, idx):
    mapping[idx]=""
    with open("test_data/" + filename, 'r') as f:
        for line in f:
            data= json.loads(line)
#             print data
            print(data['tweet']['user']['description'])
            break
getHashtag("sample10_period3.txt", 1)        

FileNotFoundError: [Errno 2] No such file or directory: 'test_data/sample10_period3.txt'

# Part 1.5

In [47]:
def dataConverttoWide(X, y):
    mod_y= []
    counter=0
    features=11
    mod_X= np.ones((1,features*6))#initialize mod_X as a row  of (5+1)*12 features
    temp=np.ones((1,features))#initialize as a row  of 12 features
    for x in X:
        counter+=1
#         print temp.shape, x.shape
        temp = np.concatenate((temp, x.reshape(1,features)),axis=1)#temp will have (6+1)*12 features
        if(counter==5):
            mod_X= np.concatenate((mod_X, temp),axis=0)
            counter=0
            temp=np.ones((1,features))#initialize as a row  of 12 features
#     final mod_X will remove first row as it was initialized and first 12 columns
    mod_X=mod_X[1:,features:]
#     create y
#     print y.shape[0]
#     print y
    for idx in range(4, y.shape[0], 5):
        mod_y.append(y[idx,0])
    mod_y= np.array(mod_y)
    print(mod_X.shape, mod_y.shape)
    return mod_X, mod_y.reshape(mod_y.shape[0], 1)

In [55]:
# datafor  1.5 b part
def getAllData():
    flag=-1
    #     df=pd.DataFrame(index=range(1), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
    #             'favoriteCount','longTweets'])

    for (htag, num_tweets) in hashtags.items():
        flag+=1
        print("Data for " + str(htag) + " hashtag")
        #prepare to read each file
        with open("tweet_data/" + "".join('tweets_#' + htag + '.txt'), 'r', encoding="utf8") as f:
            #make a df that stores the values of interest
            df1 = pd.DataFrame(index=range(num_tweets), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
            'favoriteCount','longTweets', 'rankingScore', 'userID'])
            counter = 0
            for line in f:
                #load each line and 
                tweet_data = json.loads(line)
                date = datetime.fromtimestamp(tweet_data['firstpost_date'])
                #create each line to add to the dataframe
                df1.set_value(counter, 'Time', date)
                df1.set_value(counter, 'tweetCount', 1)
                df1.set_value(counter, 'retweetCount', tweet_data['metrics']['citations']['total'])
                df1.set_value(counter, 'followerSum', tweet_data['author']['followers'])
                df1.set_value(counter, 'maxFollowers', tweet_data['author']['followers'])
                df1.set_value(counter, 'location', tweet_data['tweet']['user']['location'])
                df1.set_value(counter, 'impressionCount', tweet_data['metrics']['impressions'])
                df1.set_value(counter, 'favoriteCount', tweet_data['tweet']['favorite_count'])
                df1.set_value(counter, 'longTweet', len(tweet_data.get('title')) > 100)
                df1.set_value(counter, 'rankingScore', tweet_data.get('metrics').get('ranking_score'))
                df1.set_value(counter, 'userID', tweet_data.get('tweet').get('user').get('id'))
                counter += 1
            if flag==0:
                df = pd.concat([df1])
            else: df = pd.concat([df, df1])
    #     once all hashtags are in one data frame
    #     print df.iloc[288135,1]
    startdate=datetime(2015,2,1,8,0,0)
    enddate=datetime(2015,2,1,20,0,0)
    
    print("Before Feb. 1, 8:00 a.m.")
    dfstart = df[df.Time<startdate]
    print("Between Feb. 1, 8:00 a.m. and 8:00 p.m.")
    dfmid = df[(df.Time>startdate) & (df.Time<enddate)]
    print("After Feb. 1, 8:00 p.m")
    dfend = df[df.Time>enddate]
    
    return dfstart, dfmid, dfend
    
dfstart, dfmid, dfend = getAllData()



Data for gohawks hashtag
Data for nfl hashtag
Data for sb49 hashtag
Data for gopatriots hashtag
Data for patriots hashtag
Data for superbowl hashtag
Before Feb. 1, 8:00 a.m.
Between Feb. 1, 8:00 a.m. and 8:00 p.m.
After Feb. 1, 8:00 p.m


In [56]:
print (dfstart.shape[0]+ dfmid.shape[0]+ dfend.shape[0] )
# summ=0
# # print dfstart.shape[0]
# for htag, num_tweets in hashtags.iteritems():
#     summ+=num_tweets
    
# print summ
# print df

3138814


In [59]:
samples = {
    "sample1_period1.txt": 730,
    "sample2_period2.txt": 212273,
    "sample3_period3.txt": 3638,
    "sample4_period1.txt": 1646,
    "sample5_period1.txt": 2059,
    "sample6_period2.txt": 205554,
    "sample7_period3.txt": 528,
    "sample8_period1.txt": 229,#4 hour data given
    "sample9_period2.txt": 11311,
    "sample10_period3.txt": 365
}

In [60]:

X,y =hourly(dfstart,features)
X_start,y_start=dataConverttoWide(X, y)
lm_start=RandomForestRegressor(max_depth=features/2, random_state=0)
lm_start.fit(X_start,y_start)

X,y =hourly(dfmid,features)
X_mid,y_mid=dataConverttoWide(X, y)
lm_mid=RandomForestRegressor(max_depth=features/2, random_state=0)
lm_mid.fit(X_mid,y_mid)

X,y =hourly(dfend,features)
X_end,y_end=dataConverttoWide(X, y)
lm_end=RandomForestRegressor(max_depth=features/2, random_state=0)
lm_end.fit(X_end,y_end)

ValueError: cannot reshape array of size 12 into shape (1,11)

In [88]:
import re

for (filename,records) in samples.iteritems():
    temp=re.search('.*?(\\d+).*?(\\d+)', filename)
#     hashtag=getHashtag(filename, temp.group(1))
    groupp=temp.group(2)
#     print hashtag, groupp
    with open("test_data/" + filename, 'r') as f:
        print filename
        df = pd.DataFrame(index=range(records), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
            'favoriteCount','longTweets', 'rankingScore', 'userID'])
        counter=0
        for line in f:
                #load each line and 
                test_data = json.loads(line)
                date = datetime.fromtimestamp(test_data['citation_date'])
                #create each line to add to the dataframe
                df.set_value(counter, 'Time', date)
                df.set_value(counter, 'tweetCount', 1)
                df.set_value(counter, 'retweetCount', test_data['metrics']['citations']['total'])
                df.set_value(counter, 'followerSum', test_data['author']['followers'])
                df.set_value(counter, 'maxFollowers', test_data['author']['followers'])
                df.set_value(counter, 'location', test_data['tweet']['user']['location'])
                df.set_value(counter, 'impressionCount', test_data['metrics']['impressions'])
                df.set_value(counter, 'favoriteCount', test_data['tweet']['favorite_count'])
                df.set_value(counter, 'longTweet', test_data.get('title') > 100)
                df.set_value(counter, 'rankingScore', test_data.get('metrics').get('ranking_score'))
                df.set_value(counter, 'userID', test_data.get('tweet').get('user').get('id'))
                counter += 1
                
        X,y=hourly(df,features)
        X_test,y_test=dataConverttoWide(X, y)
        print X_test.shape ,y_test.shape
        if(groupp==1):
            y_pred=lm_start.predict(X_test)
        elif(groupp==2):
            y_pred=lm_mid.predict(X_test)
        else:
            y_pred=lm_end.predict(X_test)
        print "Prediction error MAE:", mean_absolute_error(y_test, y_pred)
        print "Prediction error RMSE:", sqrt(mean_squared_error(y_test, y_pred))

sample4_period1.txt
(1L, 55L) (1L,)
(1L, 55L) (1L, 1L)
Prediction error MAE: 1850.16333333
Prediction error RMSE: 1850.16333333
sample1_period1.txt
(15L, 55L) (15L,)
(15L, 55L) (15L, 1L)
Prediction error MAE: 194.38
Prediction error RMSE: 200.135302566
sample2_period2.txt
(4L, 55L) (4L,)
(4L, 55L) (4L, 1L)
Prediction error MAE: 20086.82
Prediction error RMSE: 38944.2048272
sample10_period3.txt
(1L, 55L) (1L,)
(1L, 55L) (1L, 1L)
Prediction error MAE: 153.6
Prediction error RMSE: 153.6
sample9_period2.txt
(3L, 55L) (3L,)
(3L, 55L) (3L, 1L)
Prediction error MAE: 510.593333333
Prediction error RMSE: 669.38634395
sample3_period3.txt
(1L, 55L) (1L,)
(1L, 55L) (1L, 1L)
Prediction error MAE: 892.027619048
Prediction error RMSE: 892.027619048
sample7_period3.txt
(1L, 55L) (1L,)
(1L, 55L) (1L, 1L)
Prediction error MAE: 110.216666667
Prediction error RMSE: 110.216666667
sample6_period2.txt
(5L, 55L) (5L,)
(5L, 55L) (5L, 1L)
Prediction error MAE: 6680.73433333
Prediction error RMSE: 14389.1406177


ValueError: Found array with 0 sample(s) (shape=(0, 55)) while a minimum of 1 is required.

In [83]:
# tetsing sample8 file
# records=229
# with open("test_data/" + "sample8_period1.txt", 'r') as f:
#     print filename
#     df = pd.DataFrame(index=range(records), columns=['Time', 'tweetCount', 'retweetCount', 'followerSum', 'maxFollowers', 'location', 'impressionCount',
#         'favoriteCount','longTweets', 'rankingScore', 'userID'])
#     counter=0
#     for line in f:
#             #load each line and 
#             test_data = json.loads(line)
#             date = datetime.fromtimestamp(test_data['firstpost_date'])
#             #create each line to add to the dataframe
#             df.set_value(counter, 'Time', date)
#             df.set_value(counter, 'tweetCount', 1)
#             df.set_value(counter, 'retweetCount', test_data['metrics']['citations']['total'])
#             df.set_value(counter, 'followerSum', test_data['author']['followers'])
#             df.set_value(counter, 'maxFollowers', test_data['author']['followers'])
#             df.set_value(counter, 'location', test_data['tweet']['user']['location'])
#             df.set_value(counter, 'impressionCount', test_data['metrics']['impressions'])
#             df.set_value(counter, 'favoriteCount', test_data['tweet']['favorite_count'])
#             df.set_value(counter, 'longTweet', test_data.get('title') > 100)
#             df.set_value(counter, 'rankingScore', test_data.get('metrics').get('ranking_score'))
#             df.set_value(counter, 'userID', test_data.get('tweet').get('user').get('id'))
#             counter += 1

#     X,y=hourly(df,features)
#     print X.shape, y.shape
# #     X_test,y_test=dataConverttoWide(X, y)

 sample8_period1.txt
(4L, 11L) (4L, 1L)


In [80]:
print y_test
print y_pred

[]
[ 5030.3]
